# Concatenador Veg loss

Se importan las librerías necesarias para manejar datos:
* pandas: buscar archivos en carpetas
* glob: trabajar con rutas y archivos del sistema
* os:  procesar cadenas con expresiones regulares
* re: Esto permite cargar, filtrar y combinar múltiples archivos CSV de manera automatizada.

In [1]:
import pandas as pd
import glob
import os
import re

Busca todos los archivos que coinciden, extrae el año de cada archivo y añade esa información como columna. Luego carga cada CSV en un DataFrame, los acumula en una lista y los concatena en un único DataFrame combinado, ordenado por año. Finalmente, guarda este dataset consolidado en un archivo CSV llamado vegetation_loss_completo_1985_2024.csv.

In [3]:
def unir_dataset_anual(nombre_base, nombre_salida):
    archivos = glob.glob(f"{nombre_base} • Annual by class • *.csv")
    archivos.sort()
    
    datos_combinados = []
    
    for archivo in archivos:
        año = extraer_año(archivo)
        if año is None or año < 1985 or año > 2024:
            continue
        df = pd.read_csv(archivo)
        df['year'] = año
        datos_combinados.append(df)
    
    if not datos_combinados:
        return None
    
    df_combinado = pd.concat(datos_combinados, ignore_index=True)
    df_combinado = df_combinado.sort_values('year').reset_index(drop=True)
    df_combinado.to_csv(nombre_salida, index=False)
    return df_combinado

def extraer_año(nombre_archivo):
    nombre_base = os.path.basename(nombre_archivo)
    patrones = [
        r'•\s*(\d{4})\.csv$',
        r'\s(\d{4})\.csv$',
        r'•.*•\s*(\d{4})\.csv$',
    ]
    for patron in patrones:
        match = re.search(patron, nombre_base)
        if match:
            año = int(match.group(1))
            if 1980 <= año <= 2030:
                return año
    return None

df_coverage_completo = unir_dataset_anual("Coverage", "coverage_completo_1985_2024.csv")
df_secondary_vegetation_completo = unir_dataset_anual("Secondary Vegetation", "secondary_vegetation_completo_1985_2024.csv")
df_vegetation_loss_completo = unir_dataset_anual("Vegetation loss", "vegetation_loss_completo_1985_2024.csv")
df_vegetation_gain_completo = unir_dataset_anual("Vegetation Gain", "vegetation_gain_completo_1985_2024.csv")